In [25]:
import requests
import re
from bs4 import BeautifulSoup, PageElement

def convert_note_name(element: PageElement):
    text = re.sub(r'\s', '', element.text)
    
    # Attempt to match single note (eg. C4)
    result = re.match(r'^[CDEFGAB][0-8]$', text)
    if result:
        return text
    
    # Attempt to match sharp note
    result = re.match(r'^([CDEFGAB])#([0-8])', text)
    if result:
        return '{}{}_#'.format(result.group(1), result.group(2))
    
    raise ValueError('invalid note name')

def convert_note_frequency(element: PageElement):
    return float(element.text)

response = requests.get('https://pages.mtu.edu/~suits/notefreqs.html')
soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')

notes = []
for row in soup.select_one('center > center > table ').select('tr'):
    children = list(filter(lambda c: c != '\n', row.contents))
    notes.append({
        'name': convert_note_name(children[0]),
        'frequency': convert_note_frequency(children[1])
    })

print('Successfully scraped {} notes'.format(len(notes)))

Successfully scraped 108 notes
